# HW1, COMS 4995_005, Deep Learning

## Max Possible Score 120 Points  (100 + 20 Extra Credits)
----

# Part1: (Basic Neural Nework) (70 Points)

### Part 1.1:

- Divide the training data into 80% training set and 20% validation set. 
- Implement the functions in the ipython notebook so that you can train your network. 
- Your code should take network structure, training data, hyperparameters and generate validation set accuracy.
- Use Relu activation for intermediate layers and use cross entropy loss after taking softmax on the output of the final layer.


Plot a graph of loss (plot loss on training set and val set both) vs Iterations (which would be decreasing with probably some noise) - **60 Points**


### Part 1.2:

Test your model accuracy on test set. If it is more than **47%**, you will get an additional score of **10 points**

# Part 2: (Regularization) (30 Points)

### Part 2.1 (15 Points) :

Modify code to add L2 regularization. Report the validation accuracy.

You should get a validation and test accuracy of more than the one reported in Part-1

Plot a graph of loss (plot loss on training set and val set both) vs Iterations (which would be decreasing with probably some noise) 


### Part 2.2 (15 Points):

You should get a validation and test accuracy crossing **50%**


# Extra Credit (20 Points)

Show your excitement on deep learning! Top **3 scorers** will get these **20 points**

**(note that you cannot use convolutional layers in this part also)**

(Hints) Boost your accuracy by trying out: 
- Dropout Regularization
- Batch Normalization
- Other optimizers like Adam
- Learning Rate Decay
- Data Augmentation 
- Different Initializations for weights like Xaviers etc.


------------

## Code Guidelines:

1. Write your code in **Python 3**.
2. **DONOT** import any other packages.
3. Click **https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz** -> download **cifar-10-python.tar.gz** -> extract as **cifar-10-python**
4. Ensure that **this ipython notebook** and **cifar-10-python** folder are in the same folder.


-------------------



## Submission Guidelines:

1. Run this ipython notebook once and submit this file. Ensure that the outputs are printed properly. We will first see the outputs, if there are no outputs, we may not run the notebook at all.
2. Training on the **test data** is considered cheating. If we find any clue of that happening, then we will disqualify the submission and it will be reported accordingly.
3. Each team member needs to separately submit the the file named uni.ipynb on courseworks.
------------

## Team Information

Team Member1 (Name,UNI): Huibo Zhao hz2480

Team Member2 (Name, UNI): Tingmeng Li tl2758

In [2]:
import numpy as np
import pickle
import copy
%matplotlib inline
import matplotlib.pyplot as plt
# Do not import other packages

In [59]:
class FullyConnectedNetwork(object):
    """
    Abstraction of a Fully Connected Network.
    Stores parameters, activations, cached values. 
    You can add more functions in this class, and also modify inputs and outputs of each function
    
    """
    def __init__(self, layer_dim, lambd=0):
        """
        layer_dim: List containing layer dimensions. 
        
        Code:
        Initialize weight and biases for each layer
        """
        
        # initialize weight and biases as dictionaries
        self.W = {}
        self.b = {}
        
        self.cacheAffine = {}
        self.cacheRelu = {}
    
        self.lambd = lambd
        
        self.num_layers = len(layer_dim)
        
        for i in range(1, self.num_layers):
            ### could try normal distribution for randomalize ###
            self.W[i] = 0.01 * np.random.randn(layer_dim[i],layer_dim[i-1]) 
            self.b[i] = 0.01 * np.random.randn(layer_dim[i],1)
            
        
        
    def feedforward(self,X):
        """
        Expected Functionality: 
        Returns output of the neural network for input X. Also returns cache, which contains outputs of 
        intermediate layers which would be useful during backprop.

        """
        
        # for layers except last layer
            # affineForward
            # relu_forward
    
        # last layer
            # affineForward 
        temp_input = X.copy()
        for i in range(1,self.num_layers):
            affine_temp_out, self.cacheAffine[i] = self.affineForward(temp_input, self.W[i],self.b[i])
            if (i == self.num_layers-1): # the last layer is output layer
                break
            temp_input = self.relu_forward(affine_temp_out)
            self.cacheRelu[i] = temp_input
        
        return affine_temp_out
            
            
    def loss_function(self, At, Y):
        """
        At is the output of the last layer, returned by feedforward.
        Y contains true labels for this batch.
        this function takes softmax the last layer's output and calculates loss.
        the gradient of loss with respect to the activations of the last layer are also returned by this function.
        
        """
        num_samples = At.shape[1] # number of samples
        
        softmax_output = np.exp(At) # do exponential to each element
        
        #print("At is",At)
        #print(softmax_output.shape)
        #print(softmax_output[:,1])

        
        temp_sum = np.sum(softmax_output,axis=0,keepdims=True) # sum(e^a[j])
        softmax_output = softmax_output / temp_sum # softmax done
        
        #print(softmax_output[:,1])
        
        # for computing the gradient of loss with respect to the activations of the last layer
        # dC/dh = y_i (prediction) - t_i (target value)
        # Therefore, we only need to minus one for those corresponding to the correct target value
        
        dAt = softmax_output.copy()
        dAt [Y,np.arange(num_samples)] -= 1
        
        # for computing average cost of all samples, we extract true class's softmax value
        # and apply -log to them and then sum them up and lastly divided by the total numbers
        ave_loss = np.sum(-np.log(softmax_output[Y,np.arange(num_samples)])) / num_samples
        
        return ave_loss, dAt

        # softmax: e^a(i) / (sum(e^a[j]) for j in all classes)
        # cross entropy loss:  -log(true class's softmax value(prediction))
    
        # for part2: when lambd > 0, you need to change the definition of loss accordingly
        
    def train(self, X, Y, max_iters=5000, batch_size=100, learning_rate=0.01, validate_every=200):
        """
        X: (3072 dimensions, 50000 examples) (Cifar train data)
        Y: (1 dimension, 50000 examples)
        lambd: the hyperparameter corresponding to L2 regularization
        
        Divide X, Y into train(80%) and val(20%), during training do evaluation on val set
        after every validate_every iterations and in the end use the parameters corresponding to the best
        val set to test on the Cifar test set. Print the accuracy that is calculated on the val set during 
        training. Also print the final test accuracy. Ensure that these printed values can be seen in the .ipynb file you
        submit.
        
        Expected Functionality: 
        This function will call functions feedforward, backprop and update_params. 
        Also, evaluate on the validation set for tuning the hyperparameters.
        """
        # generate training data index
        index = np.random.choice(50000,50000,replace=False)
        train_index = index[0:40000]
        val_index = index[40000:50000]
        
        train_X = X[:,train_index]
        train_Y = Y[:,train_index]
        
        val_X = X[:,val_index]
        val_Y = Y[:,val_index]
        
        
                
        # for some iterations
            # get current batch
            # feedforward
            # loss_function
            # backprop
            # updateParameters
        for i in range(max_iters):
            batch_X, batch_Y = self.get_batch(train_X,train_Y,batch_size)
            
            
            fdforward_output = self.feedforward(batch_X)
            
            #print(fdforward_output)
            
            loss, dAt = self.loss_function(fdforward_output,batch_Y)
            print("loss in iteration",i,"is",loss)
            
            gradients = self.backprop(loss,dAt)
            
            self.updateParameters(gradients,learning_rate)
    
    def affineForward(self, A, W, b):
        """
        Expected Functionality:
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        affine_product = W.dot(A) + b
        cache = (A,W)
        return affine_product, cache
    
    def affineBackward(self, dA_prev, cache):
        """
        Expected Functionality:
        Backward pass for the affine layer.
        dA_prev: gradient from the next layer.
        cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        
        # unpack cache
        A = cache[0]
        W = cache[1]
        
        num_samples = A.shape[1] # number of samples
        
        dA = W.T.dot(dA_prev)
        dW = dA_prev.dot(A.T) / num_samples # dW is the average dW for all samples
        db = np.mean(dA_prev,axis=1,keepdims=True) # compress each row to be its mean
        
        
        return dA, dW, db
        
    def relu_forward(self, X):
        """
        Expected Functionality:
        Forward pass of relu activation
        
        returns (A, cache)
        """
        relu_output = np.maximum(0,X)
        return relu_output
        
        
    def relu_backward(self, dx, cached_x):
        """
        Expected Functionality:
        backward pass for relu activation
        """
        deriv = cached_x.copy()
        deriv [deriv > 0] = 1
        return dx * deriv # don't use dot product
        
        
    def get_batch(self, X, Y, batch_size):
        """
        Expected Functionality: 
        given the full training data (X, Y), return batches for each iteration of forward and backward prop.
        """
        ### with or without replacement?###
        index = np.random.choice(range(X.shape[1]),batch_size,replace=False)
        
        return X[:,index], Y[:,index]
        
    def backprop(self, loss, dAct):
        """
        Expected Functionality: 
        returns gradients for all parameters in the network.
        dAct is the gradient of loss with respect to the output of final layer of the network.
        """
        gradient_W = {}
        gradient_b = {}
        
        dA, dW, db = self.affineBackward(dAct,self.cacheAffine[self.num_layers-1])
        gradient_W[self.num_layers-1] = dW
        gradient_b[self.num_layers-1] = db
        
        for i in range(self.num_layers-2,0,-1):
            relu_gradient = self.relu_backward(dA,self.cacheRelu[i])
            dA,dW,db = self.affineBackward(relu_gradient,self.cacheAffine[i])
            gradient_W[i] = dW
            gradient_b[i] = db
        
        return (gradient_W,gradient_b)
        # last layer 
            # affineBackward
            # set dW[last layer], db[last layer]
            
        # for layers except last layer
            # relu_Backward
            # affineBackward
            # set dW[layer], db[layer]
    
        # for part2: lambd > 0, you need to change the definition accordingly
    
    def updateParameters(self, gradients, learning_rate, lambd=0):
        """
        Expected Functionality:
        use gradients returned by backprop to update the parameters.
        """
        gradient_W = gradients[0]
        gradient_b = gradients[1]
        
        for i in range(1,self.num_layers-1):
            self.W[i] -= learning_rate * gradient_W[i]
            self.b[i] -= learning_rate * gradient_b[i]
        
        self.W[self.num_layers-1] -= learning_rate * gradient_W[self.num_layers-1]
        
    
    def evaluate(self, X_test, Y_test):
        '''
        X: X_test (3472 dimensions, 10000 examples)
        Y: Y_test (1 dimension, 10000 examples)
        
        Expected Functionality: 
        print accuracy on test set
        '''
        fdforward_output = self.feedforward(X_test)
        prediction = np.argmax(fdforward_output, axis = 0)
        count = 0
        for i in range(10000):
            if prediction[i] == Y_test[:,i][0]:
                count += 1
        return count/10000

In [57]:
class Loader:
    
    def unpickle(self,file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    
    def load_train_data(self):
        '''
        loads training data: 50,000 examples with 3072 features
        '''
        X_train = None
        Y_train = None
        for i in range(1, 6):
            pickleFile = self.unpickle('cifar-10-batches-py/data_batch_{}'.format(i))
            dataX = pickleFile[b'data']
            dataY = pickleFile[b'labels']
            if type(X_train) is np.ndarray:
                X_train = np.concatenate((X_train, dataX))
                Y_train = np.concatenate((Y_train, dataY))
            else:
                X_train = dataX
                Y_train = dataY

        Y_train = Y_train.reshape(Y_train.shape[0], 1)

        return X_train.T, Y_train.T

    def load_test_data(self):
        '''
        loads testing data: 10,000 examples with 3072 features
        '''
        X_test = None
        Y_test = None
        pickleFile = self.unpickle('cifar-10-batches-py/test_batch')
        dataX = pickleFile[b'data']
        dataY = pickleFile[b'labels']
        if type(X_test) is np.ndarray:
            X_test = np.concatenate((X_test, dataX))
            Y_test = np.concatenate((Y_test, dataY))
        else:
            X_test = np.array(dataX)
            Y_test = np.array(dataY)

        Y_test = Y_test.reshape(Y_test.shape[0], 1)

        return X_test.T, Y_test.T

In [40]:
X_train,Y_train = Loader().load_train_data()
X_test, Y_test = Loader().load_test_data()

print("X_Train: {} -> {} examples, {} features".format(X_train.shape, X_train.shape[1], X_train.shape[0]))
print("Y_Train: {} -> {} examples, {} features".format(Y_train.shape, Y_train.shape[1], Y_train.shape[0]))
print("X_Test: {} -> {} examples, {} features".format(X_test.shape, X_test.shape[1], X_test.shape[0]))
print("Y_Test: {} -> {} examples, {} features".format(Y_test.shape, Y_test.shape[1], Y_test.shape[0]))

X_Train: (3072, 50000) -> 50000 examples, 3072 features
Y_Train: (1, 50000) -> 50000 examples, 1 features
X_Test: (3072, 10000) -> 10000 examples, 3072 features
Y_Test: (1, 10000) -> 10000 examples, 1 features


## Part 1

In [ ]:
layer_dimensions = [3072,20,20,20,10]  # including the input and output layers  
# 3072 is the input feature size, 10 is the number of outputs in the final layer
FCN = FullyConnectedNetwork(layer_dimensions, lambd=0)
FCN.train(X_train, Y_train, max_iters=10000, batch_size=200, learning_rate=0.0001,validate_every=200)
# lambd, the L2 regularization penalty hyperparamter will be 0 for this part
y_predicted = FCN.evaluate(X_test,Y_test)  # print accuracy on test set

loss in iteration 0 is 2.30321112485
loss in iteration 1 is 2.30270268747
loss in iteration 2 is 2.30297767108
loss in iteration 3 is 2.3032225299
loss in iteration 4 is 2.30214368437
loss in iteration 5 is 2.30272352915
loss in iteration 6 is 2.30268218829
loss in iteration 7 is 2.30194281459
loss in iteration 8 is 2.30219076714
loss in iteration 9 is 2.30316042745
loss in iteration 10 is 2.30343753527
loss in iteration 11 is 2.30278060059
loss in iteration 12 is 2.30170414439
loss in iteration 13 is 2.3023740463
loss in iteration 14 is 2.30341103951
loss in iteration 15 is 2.30294289897
loss in iteration 16 is 2.30236046541
loss in iteration 17 is 2.30177102558
loss in iteration 18 is 2.30272625349
loss in iteration 19 is 2.30215804599
loss in iteration 20 is 2.30220333247
loss in iteration 21 is 2.30350825358
loss in iteration 22 is 2.3031379359
loss in iteration 23 is 2.30222076664
loss in iteration 24 is 2.30323434716
loss in iteration 25 is 2.30346525538
loss in iteration 26 is 2

loss in iteration 222 is 2.30286849151
loss in iteration 223 is 2.30221540297
loss in iteration 224 is 2.30192161755
loss in iteration 225 is 2.30232432733
loss in iteration 226 is 2.30193610104
loss in iteration 227 is 2.30307099772
loss in iteration 228 is 2.30281875184
loss in iteration 229 is 2.30188244264
loss in iteration 230 is 2.30269753767
loss in iteration 231 is 2.30196874795
loss in iteration 232 is 2.30318180885
loss in iteration 233 is 2.30254552597
loss in iteration 234 is 2.30257776261
loss in iteration 235 is 2.3021220625
loss in iteration 236 is 2.30327381355
loss in iteration 237 is 2.30283858614
loss in iteration 238 is 2.3025058973
loss in iteration 239 is 2.30276189269
loss in iteration 240 is 2.30251663847
loss in iteration 241 is 2.3027623794
loss in iteration 242 is 2.30241217452
loss in iteration 243 is 2.30253876927
loss in iteration 244 is 2.30228520097
loss in iteration 245 is 2.30331161001
loss in iteration 246 is 2.3022414723
loss in iteration 247 is 2.30

loss in iteration 436 is 2.30295118665
loss in iteration 437 is 2.30134983097
loss in iteration 438 is 2.30233278316
loss in iteration 439 is 2.30139466374
loss in iteration 440 is 2.30209302331
loss in iteration 441 is 2.30306592927
loss in iteration 442 is 2.30146313424
loss in iteration 443 is 2.30215255777
loss in iteration 444 is 2.3032733605
loss in iteration 445 is 2.30203903198
loss in iteration 446 is 2.30408926477
loss in iteration 447 is 2.30331678131
loss in iteration 448 is 2.30381154615
loss in iteration 449 is 2.30228311903
loss in iteration 450 is 2.30273177228
loss in iteration 451 is 2.30175850081
loss in iteration 452 is 2.30291618299
loss in iteration 453 is 2.30210369456
loss in iteration 454 is 2.3032694125
loss in iteration 455 is 2.30236848753
loss in iteration 456 is 2.30314088753
loss in iteration 457 is 2.30398272059
loss in iteration 458 is 2.3029109284
loss in iteration 459 is 2.30282095822
loss in iteration 460 is 2.30252523545
loss in iteration 461 is 2.3

loss in iteration 648 is 2.30326729293
loss in iteration 649 is 2.30197721018
loss in iteration 650 is 2.30315796034
loss in iteration 651 is 2.30319743766
loss in iteration 652 is 2.302136316
loss in iteration 653 is 2.30322678263
loss in iteration 654 is 2.30319333696
loss in iteration 655 is 2.30369589868
loss in iteration 656 is 2.30335114029
loss in iteration 657 is 2.30308436945
loss in iteration 658 is 2.30188345022
loss in iteration 659 is 2.30238192665
loss in iteration 660 is 2.30266734058
loss in iteration 661 is 2.30235989637
loss in iteration 662 is 2.30251099397
loss in iteration 663 is 2.30203676728
loss in iteration 664 is 2.30253218507
loss in iteration 665 is 2.30246616331
loss in iteration 666 is 2.30312952598
loss in iteration 667 is 2.30239171024
loss in iteration 668 is 2.30233079979
loss in iteration 669 is 2.30245841894
loss in iteration 670 is 2.30267197374
loss in iteration 671 is 2.30216359002
loss in iteration 672 is 2.30231468917
loss in iteration 673 is 2.

loss in iteration 862 is 2.30234080732
loss in iteration 863 is 2.30131036522
loss in iteration 864 is 2.30186576697
loss in iteration 865 is 2.30267211482
loss in iteration 866 is 2.30298810205
loss in iteration 867 is 2.30156848004
loss in iteration 868 is 2.30218611301
loss in iteration 869 is 2.30184988066
loss in iteration 870 is 2.30291402959
loss in iteration 871 is 2.30265218033
loss in iteration 872 is 2.30247625501
loss in iteration 873 is 2.3024024844
loss in iteration 874 is 2.30334061915
loss in iteration 875 is 2.30237876689
loss in iteration 876 is 2.30226295215
loss in iteration 877 is 2.30309003724
loss in iteration 878 is 2.3033359189
loss in iteration 879 is 2.30273087926
loss in iteration 880 is 2.30219735793
loss in iteration 881 is 2.3028166581
loss in iteration 882 is 2.30239517002
loss in iteration 883 is 2.30329818989
loss in iteration 884 is 2.30258053103
loss in iteration 885 is 2.30341223553
loss in iteration 886 is 2.30304584681
loss in iteration 887 is 2.3

loss in iteration 1070 is 2.30164162695
loss in iteration 1071 is 2.30226133816
loss in iteration 1072 is 2.30217671034
loss in iteration 1073 is 2.30286502485
loss in iteration 1074 is 2.3030594676
loss in iteration 1075 is 2.30170555784
loss in iteration 1076 is 2.30332914469
loss in iteration 1077 is 2.3035042686
loss in iteration 1078 is 2.30151900579
loss in iteration 1079 is 2.3018917721
loss in iteration 1080 is 2.30198846375
loss in iteration 1081 is 2.30199762587
loss in iteration 1082 is 2.30276385055
loss in iteration 1083 is 2.30318661547
loss in iteration 1084 is 2.30276460038
loss in iteration 1085 is 2.30263306666
loss in iteration 1086 is 2.30290151531
loss in iteration 1087 is 2.3020985876
loss in iteration 1088 is 2.3031894904
loss in iteration 1089 is 2.30215804402
loss in iteration 1090 is 2.30159495067
loss in iteration 1091 is 2.30159176349
loss in iteration 1092 is 2.30265369955
loss in iteration 1093 is 2.30287100696
loss in iteration 1094 is 2.30227474208
loss 

## Part 2

In [ ]:
layer_dimensions = [3072,..,.., 10]  # including the input and output layers  
# 3072 is the input feature size, 10 is the number of outputs in the final layer
FCN = FullyConnectedNetwork(layer_dimensions, lambd=0.1)
FCN.train(X_train, Y_train, max_iters=10000, batch_size=200, learning_rate=0.0001, validate_every=200)
# lambd, the L2 regularization penalty hyperparamter will not be 0 for this part
y_predicted = FCN.evaluate(X_test)  # print accuracy on test set

## Extra Credit